Конфигурация сессии, импорт модулей:

In [1]:
import pyspark
from delta import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

21/07/12 18:08:55 WARN Utils: Your hostname, artemgur-msi resolves to a loopback address: 127.0.1.1; using 192.168.1.34 instead (on interface wlp62s0)
21/07/12 18:08:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/artemgur/.local/share/virtualenvs/SummerPractice2021-X5z8IKuO/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/artemgur/.ivy2/cache
The jars for the packages stored in: /home/artemgur/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-374b454b-da06-4be0-9be9-08d887d7e143;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 214ms :: artifacts dl 7ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-

Создание простой таблицы:

In [2]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")

Чтение данных таблицы без SQL:

In [3]:
df = spark.read.format("delta").load("/tmp/delta-table/")
df.show()

+---+
| id|
+---+
|  2|
|  4|
|  1|
|  0|
|  3|
+---+



Чтение данных таблицы с помощью SQL:

In [4]:
df = spark.sql('SELECT * FROM delta.`/tmp/delta-table`')
df.show()

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8

+---+
| id|
+---+
|  2|
|  4|
|  1|
|  0|
|  3|
+---+



Создание таблицы с помощью SQL:

In [5]:
spark.sql('''CREATE OR REPLACE TABLE delta.`/tmp/delta-people` (
  name STRING,
  surname STRING)
USING DELTA''')

DataFrame[]

Создание таблицы без SQL:

In [7]:
# Doesn't work
DeltaTable.create(spark)\
  .tableName("delta_people_without_sql")\
  .addColumn("name", StringType())\
  .addColumn("surname", StringType())\
  .execute()

Удаление таблицы:


In [8]:
spark.sql("DROP TABLE delta_people_without_sql")

DataFrame[]

Добавление строк в таблицу:

In [9]:
spark.sql("INSERT INTO delta.`/tmp/delta-people` VALUES ('Василий', 'Петров')")
spark.sql("INSERT INTO delta.`/tmp/delta-people` VALUES ('Алексеев', 'Евгений')")

DataFrame[]

In [10]:
df = spark.read.format("delta").load("/tmp/delta-people/")
df.show()

+--------+-------+
|    name|surname|
+--------+-------+
|Алексеев|Евгений|
| Василий| Петров|
+--------+-------+



Чтение предыдущих версий данных:

In [11]:
df = spark.read.format("delta").option("versionAsOf", 1).load("/tmp/delta-people")
df.show()

+-------+-------+
|   name|surname|
+-------+-------+
|Василий| Петров|
+-------+-------+



Вставить в таблицу данные, не соответствующие схеме, невозможно:

In [14]:
spark.sql("INSERT INTO delta.`/tmp/delta-people` VALUES ('Романов', 'Петр', 1)")

AnalysisException: A schema mismatch detected when writing to the Delta table (Table ID: 9a637603-7c01-46c1-a069-9b79ebf81950).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, set the session configuration
spark.databricks.delta.schema.autoMerge.enabled to "true". See the documentation
specific to the operation for details.

Table schema:
root
-- name: string (nullable = true)
-- surname: string (nullable = true)


Data schema:
root
-- name: string (nullable = true)
-- surname: string (nullable = true)
-- col3: integer (nullable = true)

         

Можно добавлять constraints:

In [15]:
spark.sql('''ALTER TABLE delta.`/tmp/delta-people`
ADD CONSTRAINT not_too_long_names CHECK (length(name) <= 10)''')

DataFrame[]

Вставить в таблицу данные, не соответствующие constraints, невозможно:

In [16]:
spark.sql('''INSERT INTO delta.`/tmp/delta-people`
VALUES ('ВасилийВасилийВасилий', 'Петров')''')

21/07/12 18:12:45 ERROR Utils: Aborting task
org.apache.spark.sql.delta.schema.InvariantViolationException: CHECK constraint not_too_long_names (length(`name`) <= 10) violated by row with values:
 - name : ВасилийВасилийВасилий
	at org.apache.spark.sql.delta.schema.InvariantViolationException$.apply(InvariantViolationException.scala:50)
	at org.apache.spark.sql.delta.schema.InvariantViolationException$.apply(InvariantViolationException.scala:60)
	at org.apache.spark.sql.delta.schema.InvariantViolationException.apply(InvariantViolationException.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.delta.constraints.DeltaInvariantCheckerExec.$anonfun$doExecute$3(DeltaInvariantCheckerExec.scala:86)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeTask$1(FileFormatWriter.scala:278)
	at org.apache.spark.util.Uti

Py4JJavaError: An error occurred while calling o31.sql.
: org.apache.spark.sql.delta.schema.InvariantViolationException: CHECK constraint not_too_long_names (length(`name`) <= 10) violated by row with values:
 - name : ВасилийВасилийВасилий
	at org.apache.spark.sql.delta.schema.InvariantViolationException$.apply(InvariantViolationException.scala:50)
	at org.apache.spark.sql.delta.schema.InvariantViolationException$.apply(InvariantViolationException.scala:60)
	at org.apache.spark.sql.delta.schema.InvariantViolationException.apply(InvariantViolationException.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.delta.constraints.DeltaInvariantCheckerExec.$anonfun$doExecute$3(DeltaInvariantCheckerExec.scala:86)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:459)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.$anonfun$executeTask$1(FileFormatWriter.scala:278)
	at org.apache.s

Добавление столбца в таблицу:

In [17]:
spark.sql("ALTER TABLE delta.`/tmp/delta-people` ADD COLUMN surname_length INT AFTER name")
spark.read.format("delta").load("/tmp/delta-people").show()

+--------+--------------+-------+
|    name|surname_length|surname|
+--------+--------------+-------+
|Алексеев|          null|Евгений|
| Василий|          null| Петров|
+--------+--------------+-------+



Вычисление значений для добавленного столбца:

In [18]:
deltaTable = DeltaTable.forPath(spark, "/tmp/delta-people")
deltaTable.update(
  set = { "surname_length": expr("length(surname)") })
spark.read.format("delta").load("/tmp/delta-people").show()

+--------+--------------+-------+
|    name|surname_length|surname|
+--------+--------------+-------+
|Алексеев|             7|Евгений|
| Василий|             6| Петров|
+--------+--------------+-------+



Просмотр истории таблицы:

In [19]:
spark.sql("DESCRIBE HISTORY delta.`/tmp/delta-people`").show()

+-------+-------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|version|          timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|
+-------+-------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+
|      5|2021-07-12 18:12:57|  null|    null|              UPDATE|                  {}|null|    null|     null|          4|          null|        false|{numRemovedFiles ...|        null|
|      4|2021-07-12 18:12:53|  null|    null|         ADD COLUMNS|{columns -> [{"co...|null|    null|     null|          3|          null|         true|                  {}|        null|
|      3|2021-07-12 18:12:38|  null|    null|      ADD CONSTRAINT

VACUUM:

In [20]:
spark.sql('VACUUM delta.`/tmp/delta-people`')

Deleted 0 files and directories in a total of 1 directories.


DataFrame[path: string]